# 結合
結合するデータの大きさをなるべく小さくして、利用するメモリ量を少なくする。

In [1]:
import gc
import sys
import numpy as np
import pandas as pd

sys.path.append("../src")
from data_loader import load_hotel_reserve

customer_tb, hotel_tb, reserve_tb = load_hotel_reserve()


In [2]:
customer_tb.head()

,customer_id,age,sex,home_latitude,home_longitude
0,c_1,41,man,35.092193,136.512347
1,c_2,38,man,35.325076,139.410551
2,c_3,49,woman,35.120543,136.511179
3,c_4,43,man,43.034868,141.240314
4,c_5,31,man,35.102661,136.523797


In [3]:
hotel_tb.head()

,hotel_id,base_price,big_area_name,small_area_name,hotel_latitude,hotel_longitude,is_business
0,h_1,26100,D,D-2,43.064569,141.511397,True
1,h_2,26400,A,A-1,35.715320,139.939446,True
2,h_3,41300,E,E-4,35.281572,136.988565,False
3,h_4,5200,C,C-3,38.431293,140.795615,False
4,h_5,13500,G,G-3,33.597291,130.533872,True


In [4]:
reserve_tb.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100


## マスタテーブルの結合
`merge`関数を使う。`on`に結合キーを指定する。
- 複数の結合キーを設定したい場合`on=["key1", "key2"]`と書く
- テーブル間で列名が異なる場合は、`left_on`と`right_on`引数を利用

In [5]:
pd.merge(reserve_tb.query('people_num == 1'),
         hotel_tb.query('is_business'),
         on='hotel_id', how='inner')

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,base_price,big_area_name,small_area_name,hotel_latitude,hotel_longitude,is_business
0,r7,h_256,c_1,2017-12-29 10:38:36,2018-01-25,10:30:00,2018-01-28,1,103500,34500,C,C-1,38.237294,140.696131,True
1,r997,h_256,c_244,2016-10-15 22:47:40,2016-10-31,10:30:00,2016-11-02,1,69000,34500,C,C-1,38.237294,140.696131,True
2,r2602,h_256,c_650,2016-05-10 00:42:56,2016-05-12,11:00:00,2016-05-14,1,69000,34500,C,C-1,38.237294,140.696131,True
3,r3738,h_256,c_930,2017-04-12 09:53:00,2017-05-08,11:30:00,2017-05-09,1,34500,34500,C,C-1,38.237294,140.696131,True
4,r11,h_183,c_2,2016-11-19 12:49:10,2016-12-08,11:00:00,2016-12-11,1,29700,9900,G,G-4,33.595248,130.633567,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441,r2766,h_56,c_698,2017-04-08 16:41:45,2017-05-03,12:30:00,2017-05-06,1,95700,31900,C,C-3,38.435640,140.898354,True
442,r2983,h_56,c_753,2016-06-18 23:17:50,2016-07-13,10:00:00,2016-07-14,1,31900,31900,C,C-3,38.435640,140.898354,True
443,r2839,h_231,c_715,2016-04-06 02:51:01,2016-04-08,09:00:00,2016-04-10,1,29200,14600,A,A-3,35.914151,139.837520,True
444,r3476,h_43,c_866,2016-09-17 21:09:02,2016-10-03,09:30:00,2016-10-04,1,18100,18100,B,B-1,35.437093,139.799077,True


## 条件に応じた結合テーブルの切り替え
ホテルテーブルのすべてのホテルに対して、レコメンド候補のホテルを紐づけたデータを作成する。レコメンド候補のホテルは、同じ小値域(small_area_nameが等しい)のホテルが20件に満たない場合は、同じ大地域(big_area_nameが等しい)のホテルをレコメンド候補とする。

In [6]:
# small_area_nameごとにホテル数をカウント
small_area_mst = hotel_tb \
  .groupby(['big_area_name', 'small_area_name'], as_index=False) \
  .size().reset_index(drop=True)
small_area_mst.columns = ['big_area_name', 'small_area_name', 'hotel_cnt']

# 20件以上であればjoin_area_idをsmall_area_nameとして設定
# 20件未満であればjoin_area_idをbig_area_nameとして設定
# -1は、自ホテルを引いている
small_area_mst['join_area_id'] = \
  np.where(small_area_mst['hotel_cnt'] - 1 >= 20,
           small_area_mst['small_area_name'],
           small_area_mst['big_area_name'])

# 必要なくなった列を削除
small_area_mst.drop(['hotel_cnt', 'big_area_name'], axis=1, inplace=True)

# レコメンド元になるホテルにsmall_area_mstを結合することで、join_area_idを設定
base_hotel_mst = pd.merge(hotel_tb, small_area_mst, on='small_area_name') \
                   .loc[:, ['hotel_id', 'join_area_id']]

# 下記は必要に応じて、メモリを解放(必須ではないですがメモリ量に余裕のないときに利用)
del small_area_mst
gc.collect()

# recommend_hotel_mstはレコメンド候補のためのテーブル
recommend_hotel_mst = pd.concat([
  # join_area_idをbig_area_nameとしたレコメンド候補マスタ
  hotel_tb[['small_area_name', 'hotel_id']] \
    .rename(columns={'small_area_name': 'join_area_id'}, inplace=False),

  # join_area_idをsmall_area_nameとしたレコメンド候補マスタ
  hotel_tb[['big_area_name', 'hotel_id']] \
    .rename(columns={'big_area_name': 'join_area_id'}, inplace=False)
])

# hotel_idの列名が結合すると重複するので変更
recommend_hotel_mst.rename(columns={'hotel_id': 'rec_hotel_id'}, inplace=True)

# base_hotel_mstとrecommend_hotel_mstを結合し、レコメンド候補の情報を付与
# query関数によってレコメンド候補から自分を除く
pd.merge(base_hotel_mst, recommend_hotel_mst, on='join_area_id') \
  .loc[:, ['hotel_id', 'rec_hotel_id']] \
  .query('hotel_id != rec_hotel_id')

,hotel_id,rec_hotel_id
1,h_1,h_14
2,h_1,h_22
3,h_1,h_27
4,h_1,h_40
5,h_1,h_45
...,...,...
9659,h_220,h_226
9660,h_220,h_249
9661,h_220,h_271
9662,h_220,h_281


## 過去データの結合
予約テーブルのすべての行に、同じ顧客の2回前の予約金額の情報を付与する。  
`shift`関数を使用してデータ行を上下にずらす。

In [7]:
# customerごとにreserve_datetimeで並び替え
# groupby関数のあとにapply関数を適用することによって、groupごとに並び替える
# sort_values関数によってデータを並び替え、axisが0の場合は行、1の場合は列を並び替え
result = reserve_tb \
  .groupby('customer_id') \
  .apply(lambda group:
         group.sort_values(by='reserve_datetime', axis=0, inplace=False))

# resultはすでに、customer_idごとにgroup化されている
# customerごとに2つ前のtotal_priceをbefore_priceとして保存
# shift関数は、periodsの引数の数だけデータ行を下にずらす関数
result['before_price'] = \
  pd.Series(result.groupby('customer_id')['total_price'].shift(periods=2))

### 過去n件の合計値
予約テーブルのすべての行に、自身の行から2件前までの3回の合計予約金額の情報を付与する。過去の予約数が3回未満の場合は、値なしとする。  
データを複数のデータ集合に区切る`rolling`関数を使う。
- `min_periods`を設定することで、設定した件数以上を満たしたときのみ計算を行う。満たさない場合はNANになる。
- `center`をTrueにすると、自身のデータ行が真ん中になるようにWindowを選択する。設定しない場合は、自身のデータ行含めて設定した件数になるよう下のデータ行を加える。
- `rolling`関数のあとの集約関数を変更することでさまざまな計算ができる。max/min/meanなど

In [8]:
# customer_idごとにreserve_datetimeでデータを並び替え
result = reserve_tb.sort_values(by="customer_id")\
  .groupby("customer_id")\
  .apply(lambda x: x.sort_values(by="reserve_datetime", ascending=True))\
  .reset_index(drop=True)

In [9]:
result

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100
...,...,...,...,...,...,...,...,...,...
4025,r4024,h_244,c_999,2016-10-06 18:01:34,2016-10-07,11:30:00,2016-10-09,3,59400
4026,r4025,h_160,c_999,2017-03-11 11:56:05,2017-03-27,10:00:00,2017-03-30,1,37200
4027,r4026,h_129,c_999,2017-06-27 23:00:02,2017-07-10,09:30:00,2017-07-11,2,16000
4028,r4027,h_97,c_999,2017-09-29 05:24:57,2017-10-09,10:30:00,2017-10-10,2,41800


In [10]:
result["price_sum"] = pd.Series(
    result\
    .groupby("customer_id")["total_price"].rolling(center=False, window=3, min_periods=3).sum()\
    .reset_index(drop=True)
)

In [11]:
# 特定のcustomer_idでチェック
result.query('customer_id=="c_7"')

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,price_sum
2619,r39,h_232,c_7,2016-03-21 16:14:52,2016-04-10,10:00:00,2016-04-11,2,57000,NaN
2620,r40,h_102,c_7,2016-09-01 01:19:57,2016-09-17,11:30:00,2016-09-18,3,18000,NaN
2621,r41,h_23,c_7,2016-10-14 02:41:13,2016-10-15,12:00:00,2016-10-16,2,130200,205200.0
2622,r42,h_63,c_7,2016-11-11 12:42:32,2016-11-26,09:00:00,2016-11-29,1,44700,192900.0
2623,r43,h_224,c_7,2017-04-22 20:03:46,2017-05-22,11:00:00,2017-05-25,3,36000,210900.0
2624,r44,h_145,c_7,2017-06-23 23:56:10,2017-07-18,09:30:00,2017-07-21,2,112800,193500.0
2625,r45,h_104,c_7,2017-09-28 11:44:02,2017-10-05,09:00:00,2017-10-07,1,84400,233200.0


## 過去n件の平均値
予約テーブルのすべての行に、自身の行を含めないで1件まえから3件前までの3回の平均予約金額の情報を付与する。過去の予約が3回未満の場合、満たない回数内で平均予約金額を計算する。予約が1回もない場合は、値なしとする。

In [12]:
reserve_tb['price_avg'] = pd.Series(
  reserve_tb

    # データ行をcustomer_idごとにグループ化
    .groupby('customer_id')

    # customer_idごとにreserve_datetimeでデータを並び替え
    .apply(lambda x: x.sort_values(by='reserve_datetime', ascending=True))

    # total_priceのwindow3件にまとめ、その平均値を計算
    # min_periodsを1に設定し、1件以上あった場合には計算するよう設定
    ['total_price'].rolling(center=False, window=3, min_periods=1).mean()

    # group化を解除すると同時に、customer_idの列を削除
    .reset_index(drop=True)
)

# customer_idごとにprice_avgを1行下にずらす
reserve_tb['price_avg'] = \
  reserve_tb.groupby('customer_id')['price_avg'].shift(periods=1)

In [13]:
reserve_tb.query('customer_id=="c_87"')

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,price_avg
372,r373,h_137,c_87,2016-01-29 21:21:31,2016-02-18,12:30:00,2016-02-21,4,74400,NaN
373,r374,h_54,c_87,2016-03-24 19:59:30,2016-04-17,12:30:00,2016-04-18,2,25600,71333.333333
374,r375,h_219,c_87,2016-06-22 21:56:52,2016-06-26,11:30:00,2016-06-28,4,82400,55866.666667
375,r376,h_45,c_87,2016-09-23 21:00:03,2016-10-22,12:00:00,2016-10-24,3,60000,60800.000000
376,r377,h_275,c_87,2016-11-25 06:28:06,2016-11-28,10:30:00,2016-12-01,2,111600,56000.000000
377,r378,h_109,c_87,2017-01-06 06:29:47,2017-01-25,11:00:00,2017-01-28,2,63000,84666.666667
378,r379,h_8,c_87,2017-05-03 00:04:07,2017-05-29,12:00:00,2017-06-01,3,111600,78200.000000
379,r380,h_196,c_87,2017-07-14 22:04:48,2017-08-01,09:00:00,2017-08-04,1,70500,95400.000000


## 過去n日間の合計値
予約テーブルのすべての行に対して、自身の行を含めないで同じ顧客の過去90日間の合計予約金額の情報を付与する。予約が1回もない場合は0とする。

`operator`ライブラリの`and_`関数は引数にbooleanリストのリストを指定し、リストのインデックスごとにすべてTrueのときのみTrueに、それ以外はFalseに変換する関数。

operator.and_([True, False, True, False], [True, True, False, False])の場合、[True, False, False, False]が返される


In [14]:

import pandas.tseries.offsets as offsets
import operator

# 日時の計算に利用するため、データ型を文字列から日付型に変換
# (「第10章 日時型」で詳しく解説)
reserve_tb['reserve_datetime'] = \
  pd.to_datetime(reserve_tb['reserve_datetime'], format='%Y-%m-%d %H:%M:%S')

# reserve_datetimeの日付を確認せずに、同じcustomer_idのデータ行同士をすべて結合
sum_table = pd.merge(
	reserve_tb[['reserve_id', 'customer_id', 'reserve_datetime']],
  reserve_tb[['customer_id', 'reserve_datetime', 'total_price']]
            .rename(columns={'reserve_datetime': 'reserve_datetime_before'}),
  on='customer_id')

# checkinの日付を比較して、90日以内のデータが結合されているデータ行のみ抽出
# operatorのand_関数を利用して、複合条件を設定
# reserve_idごとにtotal_priceの合計値を計算
# (日付のデータ型については、「第10章 日時型」で詳しく解説)
sum_table = sum_table[operator.and_(
  sum_table['reserve_datetime'] > sum_table['reserve_datetime_before'],
  sum_table['reserve_datetime'] + offsets.Day(-90) <= sum_table['reserve_datetime_before']
)].groupby('reserve_id')['total_price'].sum().reset_index()

# 列名を設定
sum_table.columns = ['reserve_id', 'total_price_sum']

# 計算した合計値を結合し、元のテーブルに情報を付与
# 合計値が存在しないレコードの合計値の値を、fillnaを利用して0に変更
pd.merge(reserve_tb, sum_table, on='reserve_id', how='left').fillna(0)

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,price_avg,total_price_sum
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,0.000000,0.0
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,97200.000000,0.0
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,58900.000000,20600.0
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,50466.666667,0.0
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,82866.666667,0.0
...,...,...,...,...,...,...,...,...,...,...,...
4025,r4026,h_129,c_999,2017-06-27 23:00:02,2017-07-10,09:30:00,2017-07-11,2,16000,53000.000000,0.0
4026,r4027,h_97,c_999,2017-09-29 05:24:57,2017-10-09,10:30:00,2017-10-10,2,41800,37533.333333,0.0
4027,r4028,h_27,c_999,2018-03-14 05:01:45,2018-04-02,11:30:00,2018-04-04,2,74800,31666.666667,0.0
4028,r4029,h_48,c_1000,2016-04-16 15:20:17,2016-05-10,09:30:00,2016-05-13,4,540000,0.000000,0.0


## 全結合
全結合とは、結合するテーブル同士のすべての組み合わせをかけ合わせて生成する結合。  
顧客ごとに2017年1月～2017年3月の月間合計利用料金を計算する。利用がない月は0とする。

In [15]:
# 日付型用のライブラリ
import datetime
# 日付の計算用のライブラリ
from dateutil.relativedelta import relativedelta

# 年月マスタの生成
month_mst = pd.DataFrame({
  'year_month':
    # relativedeltaで2017-01-01をx月間進める、xは0,1,2を代入
    # 2017-01-01, 2017-02-01, 2017-03-01のリストを生成
    [(datetime.date(2017, 1, 1) + relativedelta(months=x)).strftime("%Y%m")
     for x in range(0, 3)]
})

# cross joinのためにすべて同じ値の結合キーを準備
customer_tb['join_key'] = 0
month_mst['join_key'] = 0

# customer_tbとmonth_mstを準備した結合キーで内部結合し、全結合を実現
customer_mst = pd.merge(
  customer_tb[['customer_id', 'join_key']], month_mst, on='join_key'
)

# 年月の結合キーを予約テーブルで準備
reserve_tb['year_month'] = reserve_tb['checkin_date'] \
  .apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d').strftime("%Y%m"))

# 予約レコードと結合し、合計利用金額を計算
summary_result = pd.merge(
  customer_mst,
  reserve_tb[['customer_id', 'year_month', 'total_price']],
  on=['customer_id', 'year_month'], how='left'
).groupby(['customer_id', 'year_month'])["total_price"] \
 .sum().reset_index()

# 予約レコードがなかった場合の合計金額を値なしから0に変換
summary_result.fillna(0, inplace=True)

In [16]:
summary_result

,customer_id,year_month,total_price
0,c_1,201701,0.0
1,c_1,201702,0.0
2,c_1,201703,194400.0
3,c_10,201701,0.0
4,c_10,201702,0.0
...,...,...,...
2995,c_998,201702,0.0
2996,c_998,201703,0.0
2997,c_999,201701,0.0
2998,c_999,201702,0.0
